# Mini Problem Set – Trajectory Optimization for Under-actuated Robots

### Due May 10, 2017 at 23:59:59

Be sure to load the modules below

In [13]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import scipy.linalg
from numpy import matrix
from IPython.display import YouTubeVideo, HTML
from utils import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


---
## Part 1 - Manual Control Gains (x points)

In this problem set, you will learn to control the cart-pole, a simple underactuated system that illustrates the process of solving and the methods required to control underactuated systems.

The cart-pole is a classic control problem, as the dynamics of the system are relatively straightforward, but the control of the cart-pole is inherently complex due its under-actuated nature. Remember from lecture that “under-actuated” is another way of saying that there are less controllable actuators than there are degrees of freedom.

---
## The Cart-Pole

Below, you will implement the control system for the cart-pole, the canonical underactuated system. See the figure below for an idea of what this system looks. 

![](images/cart-pole.png)
<emph>a cart with a pendulum (the pole) attached to it. The cart is constrained to lateral movement, and the objective is to balance the pendulum at the top of the cart. Describe what the degrees of freedom are</emph>

As you saw in lecture, the cart-pole is a system with two degrees of freedom, but only one degree of freedom is actuated. Therefore, this system is under-actuated.  
Here, we show the cart-pole system in free-fall, with no actuated control. As you can see, the system is unstable.

In [14]:
play_local_video('trimmed_freefall')

You will learn how to stabilize the pendulum by controlling the cart’s lateral movement. The goal of this problem is to reach a stable final state, in which the pendulum is perfectly balanced on above of the cart’s center of mass at its initial starting position on the x axis, after starting from any initial state. 
You will implement helper functions and adjust the control parameters of the system.
Just a note, the coordinate system we’re using here is different from the cart-pole demo presented in lecture. In our problem set, our unstable equilibrium state with the pole centered at the top of the cart is $\theta = 0$, while the pole centered below the cart is $\theta = \pi$. $\theta$ is positive in the clockwise direction.

## Manually Find the Control Gains (K Matrix) (15 points):

As we saw in the free-fall video, balancing the pendulum at the top of the cart’s center of mass is an unstable equilibrium. Any perturbation to the cart-pole system will move the pendulum away from this unstable equilibrium point. 

We must therefore devise a control system that drives the pendulum to its point of unstable equilibrium, and stabilizes the pendulum on top of the cart. 

To accomplish this, we will use a closed loop full state feedback controller. 

We will explain the feedback controller as follows:

Our state vector for the system is $[x, dx, \theta, d\theta]$ (note that this is configured differently from the lecture Matlab demo vector). This state vector is multiplied by a feedback gain of -K and fed back as the input force to the side of the cart in the positive x direction, such that $u = -Kx$.

In this section, you will use a tried and true method for finding the K matrix - you will tune the matrix by hand. Play around with the K matrix values until you find a gain vector K that successfully keeps the pole upright.

You can learn in depth about the theory behind the feedback controller from [these notes](https://ocw.mit.edu/courses/aeronautics-and-astronautics/16-30-feedback-control-systems-fall-2010/lecture-notes/MIT16_30F10_lec11.pdf), but don’t spend too much time on this section. The main goal here is to provide an intuition for what the K gain matrix represents, and how manipulating the matrix leads to control over the system.

In [6]:
def manualGains():
    """
    Try setting different values for K matrix and 
    running the next cell to see the resulting behaviour

    The resulting plot shows the x (red trace) and theta (green trace) displacement over time. 
    """
    manual_K = matrix([[0., 0., 0., 0.]])
    ### BEGIN SOLUTION
    # A good example of for a K that stabilizes the system is 
    manual_K = matrix([[0., -2, -40., -7.]])
    ### END SOLUTION    
    return manual_K

In [7]:
K = manualGains()
control_with_manual_gain(K)

NameError: name 'x_0' is not defined

Congratulations! You have successfully controlled the behaviour of the system using a simple hand-tuned gain matrix. Unfortunately, in most real system the number of possible states increases, and it can be very hard to build an intuition on how the K matrix will affect the the control system behaviour. Also, hand-tuned gains are not necessarily optimal. 

Fortunately, we live in the 21st century where we can have computers run algorithms that solve these type of problems quite nicely. This is where the next section comes into play - we will use the linear quadratic regulator (LQR) to compute the optimal feedback gain K matrix!

---

## Optimally find the Control Gains using LQR (40 points total):

Using LQR, we will pose the control problem as an optimization problem in which we aim to minimize a cost function that is a combination of a current state cost, an input cost, and (optionally) a final state cost. LQR is guaranteed to find the optimal solution for the cost function provided. 

The only problem, is that as LQR’s name might already give away, it requires the system to be linear. The cart-pole system, on the other hand, is non-linear. So how do we use LQR to optimize the controls for our system? By linearizing the system!

Linearization of the system is taken care of by understanding the dynamics of the systems, using Taylor expansions on those dynamics, and taking the first order parts of the Taylor expansions as the linearization. These are the first order Taylor expansions of the manipulator equations we briefly explored in lecture. Don’t worry, you won’t have to derive the dynamics yourself for this mini problem set. However, if you want to learn more about how the linearization  is derived, check out [section 3.3.1](http://underactuated.csail.mit.edu/underactuated.html?chapter=3) of MIT’s Underactuated Robotics course lecture notes. 

The dynamics of the cart-pole system are “relatively” simple to derive, compared to other more complicated under-actuated systems. 

Below is an example matrix linearization of the cart-pole system, derived using the Lagrangian mechanics of the kinetic energy of the cart and the pendulum, as well as the inertia of the pendulum. Matrix A is the state matrix, and B is the input matrix.

![](images/linearizedAB.png)

The representation of the linear system is then:
$ x’ =Ax+Bu $


The dynamics of most under-actuated systems are unique to the mechanics of each system. Thus, while it is important to know how to linearize the system by calculating the dynamics, deriving the dynamics is outside the scope of this problem set. Instead, we have taken care of this for you, and pass in the appropriate linearized A and B matrices to the LQR method you will implement below. 

## Implementing LQR (20 points):

As our system has been linearized, we can use LQR to solve for the gains matrix K. Recall that LQR finds the optimal solution for a given quadratic cost function by minimizing the value of the cost.  

The feedback control law that minimizes the value of the cost (LQR):

$u^{*} = - R^{-1} B^T S x = -Kx$

Recall that in lecture we talked about the Algebraic Riccati Equation (ARE) that is used to solve for S, which in turn is used to solve for K. 

Algebraic Ricatti Equation:

$0 = SA + A^T S - SBR^{-1}B^T S + Q$

Below, you will implement LQR using the given equations. You will most likely find [this API reference](https://docs.scipy.org/doc/scipy-0.19.0/reference/linalg.html) page useful for solving the continuous Algebraic Riccati Equation solution and other linear algebra operations such as computing the inverse or transpose.

In [ ]:
def solve_lqr(A, B, Q, R):
    """
    Solve the continuous time LQR controller
    Inputs - A, B, Q, R
    A: State matrix
    B: Input matrix
    Q: The gain of the cost function on the states of the system R matrix
    R: gain of the cost function on the input to the system.
    Returns - K: Gains matrix
    """
    ### BEGIN SOLUTION
    S = matrix(scipy.linalg.solve_continuous_are(A, B, Q, R))
    K = matrix(scipy.linalg.inv(R)*(B.T*S))
    return K
    ### END SOLUTION

In [36]:
check_lqr_solver(solve_lqr)

NameError: name 'check_lqr_solver' is not defined

The unstable equilibrium state vector of the this system is $u = [x_0, dx_0, \theta_0, d\theta_0] = [0, 0, 0, 0]$. Note that $\theta = 0$ denotes the pole’s upright position relative to the cart’s center of mass.

---
## Choosing Q and R Cost Matrices (20 points)

Now, we define two Q and R matrices. The Q matrix is the gain of the cost function on the states of the system, while the R matrix is the gain of the cost function on the input to the system. 

Below, you will fill in valid values for the Q and R matrices. Your Q and R matrices are considered ‘valid’ if they return the cart-pole system back to our final goal state. Remember that our main goal state is the keep the pole upright (theta = 0), and to drive the cart back to it’s initial position (x=0). Additionally, we want our input to be as minimal as possible. Therefore, we will implement these goals into our control system using the Q and R matrices.

In [42]:
def tuneQandR():
    """
    Tune the Q and R matrices
    """
    Q = matrix([
        [1, 0, 0, 0],
        [0, 1, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1]
    ])
    R = 10
    ### BEGIN SOLUTION
    # For instance...
    Q = matrix([
        [100, 0, 0, 0],
        [0, 1, 0, 0],
        [0, 0, 100, 0],
        [0, 0, 0, 1]
    ])
    R = 6
    ### END SOLUTION    
    return (Q, R)

In [7]:
(Q, R) = tuneQandR()
tune_cost_functions(Q, R)

NameError: name 'matrix' is not defined

Congratulations! You have now successfully implemented the LQR solver.

---
## Limitations of LQR (15 points total)

Unfortunately, as we use LQR with the linearization of a nonlinear system around a fixed end goal state, LQR is only an optimal policy when the system is already in the range of initial values near the vicinity of the fixed goal state. The range of initial values for which LQR will stabilize the system was introduced in lecture as the basin of attraction. For the cart-pole system, the dimensions of the basin of attraction are all the initial $\theta$ and $d\theta$ values for which LQR converges to the goal state.

Therefore, LQR works well as a stabilizer for the cart-pole system when the pendulum is at an angle near the top of the cart’s center of mass and has a small enough angular speed, or in other words is inside the basin of attraction. LQR does not converge to the goal state when the pendulum is in a state that is outside the basin of attraction.

In [15]:
play_local_video('lqr_no_swingup_pi')

In this method, you will test the size of LQR’s basin of attraction by varying the initial $\theta_{init}$ of the system. We will not worry about the $d\theta$ dimension for this problem. 

Play with the system as much as you can until you find the highest ${\pm\theta_{init}}$ threshold value for which LQR alone is still able to stabilize the system above the cart’s center of mass. We will be using a tolerance to test this problem, so don’t worry too much about finding the exact $\theta$ value LQR fails at.

In [19]:
def test_lqr_limits():
    """
    Try different ranges for the initial pendulum angle
    outputs: (neg_threshold, pos_threshold)
    - pos_threshold: theta initial (in radians)
    - neg_threshold: - theta initial
    """
    pos_threshold = pi/2
    neg_threshold = -pi/2
    
    return (neg_threshold, pos_threshold)

In [ ]:
(neg_threshold, pos_threshold) = test_lqr_limits()
test_threshold(neg_threshold, pos_threshold)

As LQR fails when the initial state of the system is outside of its basin of attraction, we must instead devise a nonlinear control that is able to drive the system near the unstable equilibrium point that LQR is linearized at. We have already implemented this controller for you, called the “swing-up controller”. The swing-up controller is derived using the energy of pendulum and a linearization of the actuated cart counteracting the dynamic contributions of the pendulum (called the collocated partial feedback linearization). If you’re interested in learning more about the derivation of the swing-up controller, feel free to read [sections 3.4-3.5](http://underactuated.csail.mit.edu/underactuated.html?chapter=3) of the lecture notes.

The swing-up is a nonlinear controller that will push all system trajectories to the unstable equilibrium. But does it make the unstable equilibrium locally stable? No. Small perturbations may cause the system to drive all of the way around the circle in order to once again return to the unstable equilibrium, as shown in the video below.
%%HTML
<video width="320" height="240" controls>
  <source src=".media/no_lqr_dx0-01.mp4" type="video/mp4">
</video>

For this reason, once trajectories come into the vicinity of our swing-up controller, we must switch to our LQR balancing controller to complete the task.  

In the demo below, we show that even when the initial $\theta$ values of the system are outside the threshold you found above, the swing-up controller is able to kick in and eventually push the system back into a state inside LQR’s basin of attraction, at which point the system switches to LQR control in order to balance the pendulum at the unstable equilibrium point.

In [ ]:
simulate_with_swingup()

---
## LQR Trees (30 points total)

We learned in lecture that LQR trees are a method of extending the linearized control of underactuated systems. 

Remember that in the previous section, we showed that LQR is inadequate when the states of the system are outside LQR’s linearized point’s basin of attraction. LQR-trees work around this limitation by linearizing the dynamics of the system around a set of points and running LQR on each of these points to generate a unique K gains matrix for each point. Thus, the basin of attraction of the system grows as each of the linearized points has its own basin of attraction, and the control optimization of each of the points ‘pulls’ the system as a whole to its intended final state.

For the following questions, [this paper](http://groups.csail.mit.edu/robotics-center/public_papers/Tedrake09a.pdf) is a useful resource, if necessary.

### Short answer 1 (10 points):

If the cart-pole system were implemented using LQR-trees, how would the basin of attraction compare to the system we implemented? The video below of a the cart-pole implemented using LQR trees.

In [37]:
from IPython.display import YouTubeVideo
YouTubeVideo('Bzq96V1yN5k')

[YOUR ANSWER HERE]

### Short answer 2 (10 points):

Would you still need the swing-up controller? Why or why not?

[YOUR ANSWER HERE]

### Short answer 3 (10 points):

What would you expect about the computational difficulty of LQR-trees vs the system we implemented? Explain.

[YOUR ANSWER HERE]